In [ ]:
!pip install conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-train.conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-test.conllu
!pip install conllu nltk scikit-learn xgboost


--2024-08-18 00:08:11--  http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-train.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.149.154
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.149.154|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7444078 (7.1M)
Saving to: ‘pt_porttinari-ud-train.conllu.1’

pt_porttinari-ud-tr 100%[===================>]   7.10M   317KB/s    in 22s     

2024-08-18 00:08:34 (329 KB/s) - ‘pt_porttinari-ud-train.conllu.1’ saved [7444078/7444078]

--2024-08-18 00:08:34--  http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-test.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.149.154
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.149.154|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2120477 (2.0M)
Saving to: ‘pt_porttinari-ud-test.conllu’

pt_porttinari-ud-te 100%[===================>]   2.02M   355KB/s    in 7.3s    

2024-08-18 00:08:4

Importação das libs

In [ ]:
import conllu
import itertools as it
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, DMatrix
from sklearn.metrics import accuracy_score


# Extração de dados

Funções

In [ ]:

class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class CoNLLU:
    def __init__(self, files):
        self.words = []
        self.sentences = []
        for f in files:
            with open(f, encoding='utf-8') as file:
                parsed = conllu.parse(file.read())
                sents = [[AttributeDict(form=token['form'], lemma=token['lemma'], pos=token['upos'], feats=token['feats'])
                          for token in tokenlist if token['upos'] != '_']
                         for tokenlist in parsed]
                self.sentences.extend(sents)
                self.words.extend([word for sent in sents for word in sent])
        self.pos_tags = set([word.pos for word in self.words])
        self.feats_dict = {pos: set(it.chain.from_iterable([list(word.feats.keys())
                                                           for word in self.words if word.pos == pos and word.feats is not None]))
                           for pos in self.pos_tags}

# Função para extrair as features de uma palavra
def extract_features(tokens, index, history):
    word = tokens[index][0]  # Extraímos a palavra da tupla (word, pos)
    features = {
        'word': word.lower(),
        'suffix-1': word[-1:],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
        'prefix-1': word[:1],
        'prefix-2': word[:2],
        'prefix-3': word[:3],
        'is_capitalized': word[0].upper() == word[0],
        'is_all_caps': word.upper() == word,
        'is_all_lower': word.lower() == word,
        'prev_word': '' if index == 0 else tokens[index - 1][0].lower(),  # Extraindo a palavra anterior da tupla
        'prev_tag': '' if index == 0 else history[index - 1],
        'next_word': '' if index == len(tokens) - 1 else tokens[index + 1][0].lower(),  # Extraindo a próxima palavra da tupla
    }
    return features


Manipulando dados


In [ ]:
# Carregar o dataset
bosque = CoNLLU(files=["pt_porttinari-ud-train.conllu","pt_porttinari-ud-test.conllu"])

# Preparar as sentenças com as tags
tagged_sents = [[(word.form, word.pos) for word in sent] for sent in bosque.sentences]

# Criar o conjunto de dados (features, label)
data = [(extract_features(sent, i, sent[:i]), sent[i][1]) for sent in tagged_sents for i in range(len(sent))]

# Separar características e rótulos
X, y = zip(*data)
vectorizer = DictVectorizer(sparse=True)
X_vec = vectorizer.fit_transform(X)

# Codificar rótulos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir os dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_vec, y_encoded, test_size=0.2, random_state=42)

# Treinamento



In [ ]:

# Criar DMatrix para o treinamento e teste
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

# Definir o modelo
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Treinar o modelo
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:14:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

# Avaliando modelo

In [ ]:

# Avaliar o modelo no conjunto de teste
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy * 100:.2f}%")



Acurácia no conjunto de teste: 95.91%


# Utilizando o classifierbasedtagger com o naivebayes.

In [ ]:
import conllu
import itertools as it
from nltk.classify import NaiveBayesClassifier
from nltk.tag import ClassifierBasedTagger
from sklearn.model_selection import train_test_split

# Classe AttributeDict para tratar as palavras como atributos
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

# Classe para manipular arquivos CoNLLU
class CoNLLU:
    def __init__(self, files):
        self.words = []
        self.sentences = []
        for f in files:
            with open(f, encoding='utf-8') as file:
                parsed = conllu.parse(file.read())
                sents = [[AttributeDict(form=token['form'], lemma=token['lemma'], pos=token['upos'], feats=token['feats'])
                          for token in tokenlist if token['upos'] != '_']
                         for tokenlist in parsed]
                self.sentences.extend(sents)
                self.words.extend([word for sent in sents for word in sent])
        self.pos_tags = set([word.pos for word in self.words])
        self.feats_dict = {pos: set(it.chain.from_iterable([list(word.feats.keys())
                                                           for word in self.words if word.pos == pos and word.feats is not None]))
                           for pos in self.pos_tags}

# Função para extrair as features de uma palavra
def extract_features(tokens, index, history):
    word = tokens[index][0]
    features = {
        'word': word.lower(),
        'suffix-1': word[-1:],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
        'prefix-1': word[:1],
        'prefix-2': word[:2],
        'prefix-3': word[:3],
        'is_capitalized': word[0].upper() == word[0],
        'is_all_caps': word.upper() == word,
        'is_all_lower': word.lower() == word,
        'prev_word': '' if index == 0 else tokens[index - 1][0].lower(),
        'prev_tag': '' if index == 0 else history[index - 1],
        'next_word': '' if index == len(tokens) - 1 else tokens[index + 1][0].lower(),
    }
    return features

# Carregar o dataset
bosque = CoNLLU(files=["pt_porttinari-ud-train.conllu","pt_porttinari-ud-test.conllu"])

# Preparar as sentenças com as tags
tagged_sents = [[(word.form, word.pos) for word in sent] for sent in bosque.sentences]

# Dividir os dados para treino e teste
train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)

# Definir o classificador do NLTK
nltk_classifier = NaiveBayesClassifier.train(
    [(extract_features(sent, i, sent[:i]), sent[i][1]) for sent in train_sents for i in range(len(sent))]
)

# Criar o tagger usando o classificador treinado
tagger = ClassifierBasedTagger(train=train_sents, feature_detector=extract_features)

# Avaliar o desempenho no conjunto de teste
accuracy = tagger.evaluate(test_sents)
print(f"Acurácia no conjunto de teste: {accuracy * 100:.2f}%")

# Aplicar o tagged-based tagger
def tag_sentence(sentence):
    tokens = sentence.split()
    tagged_sentence = tagger.tag(tokens)
    return tagged_sentence


<ipython-input-12-479ca70780ab>:69: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = tagger.evaluate(test_sents)


Acurácia no conjunto de teste: 60.30%
